<div style="text-align: left">
 <img src="./graph/LogoParisSaclay.png" width="850px">
</div>

#  <a style="color:#920037"> DataCamp Project - Tennis Match Prediction </a>

### Authors : Bilal MELLOUKI - Aimane EL MAIMOUNI

#### M2 Data Science - Université Paris Saclay  
 
**Professors :** Alexandre GRAMFORT & Balazs KEGL

# <a style="color:#920037"> 1. TASK EXPLANATION </a>

In this challenge, we will try to predict if we will gain revenue or not while displaying an add video, the data contains some displayed videos and the revenue gained from them.

The strategy of calculating the revenue is : If the video is displayed, then we will incur a fixed cost. However, we will benefit from a revenue only if the video is watched a minimum number of seconds by the user.

Thus, there are three possible cases:

* We do not display the video: Cost=0, Revenue=0, Profit=0
* We display a video that is not watched long enough: Cost>0, Revenue=0, Profit=-Cost
* We display a video that is watched long enough: Cost>0, Revenue>0, Profit=Revenue-Cost

Our data is a csv file of 100000 examples of video ad displays. It contains the following columns:

* creative_id: A unique identifier of the video that has been displayed to the user
* user_operating_system: The user Operating System (OS)
* user_device: The user device type
* average_seconds_played: The average number of seconds the user usually watches our videos (only if we already know the user, based on the user history)
* cost: The cost we had to pay to display the video
* revenue: The revenue generated by this video when it has been watched

# <a style="color:#920037"> 1. DATA exploration </a>
## <a style="color:#920037"> 1.1 Data downloding </a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.impute import SimpleImputer 
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [ ]:
#download the data
%run download_data.py

In [ ]:
#data extraction :
data = pd.read_csv("data/aft100k.csv.zip",compression='zip')

## <a style="color:#920037"> 1.2 Some informations from the data  </a>

#### What is our global margin based on this data ?

In [ ]:
print("Our global margin based on this log is : ",(data.revenue.sum()-data.cost.sum())/data.revenue.sum())

#### What are the most profitable Operating Systems ?

In [ ]:
#adding a colomn indicating profit for each video
data.profit = data.revenue-data.cost
data.insert(6,"profit",data.profit)

In [ ]:
#the most profitable Operating Systems in a descending order are:
pd.DataFrame(data.groupby(["user_operating_system"]).profit.sum().sort_values(ascending=False))

In [ ]:
#drop the profit column:
data.drop(columns=["profit"],axis=0,inplace=True)

## <a style="color:#920037"> 1.2 Some Graphs  </a>

# <a style="color:#920037"> 2. DATA PREPROCESSING AND MODEL TESTING</a>
## <a style="color:#920037"> 2.1 visualisation And Some Graphs </a>

In [ ]:
#what the data looks like :
data.sample(10)

In [ ]:
#data columns:
data.columns

In [ ]:
#the number of missing values for each column
data.isna().sum()

###### We see that the column user machine has only 8 missing values, so we are going to delete the rows corresponding to these ones.

In [ ]:
#dropping this Nan values
data.dropna(subset=["user_device"],axis=0,inplace=True)

In [ ]:
#checking the result:
data.isna().sum()

In [ ]:
#the distibution of cost and revenue together:
plt.figure(figsize=(15,10))
k = sns.distplot(data.cost,hist=False,label = "cost density")
k = sns.distplot(data.revenue,hist=False,label = "revenue density")

In [ ]:
#the distibution of average_seconds_played:
plt.figure(figsize=(15,10))
k = sns.distplot(data.average_seconds_played,hist=False,label = "average_seconds_played density")

As we can see, the revenue mode is lacated at 0; the majority of revenue are arround 0, cost is approximately never takes the zero as value,which is logical since the video that was displayed has a cost higher than zero.

Also there average_seconds_played has a mode located at 0, like for revenue, the same conclusion can be applied.

##### Now let's display  values of  user_device and user_operating_system, such that the font size of each value is proportional to its frequency.

In [ ]:
#For user_device
text = " ".join(data.user_device)  
wordcloud = WordCloud(width=480, height=480,colormap="Blues",max_words=50).generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#For user_operating_system
text = " ".join(data.user_operating_system)  
wordcloud = WordCloud(width=480, height=480,colormap="Blues",max_words=50).generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## <a style="color:#920037"> 2.2 Columns transformation and performance measure:</a>

### Phase 1 : drop the missing value 

As indicated, for now, we will drop all the rows containing missing values

In [ ]:
#we create a copy of our data so that changes we will always have the original data for other preprocessing techniques
new_data = data.copy()

In [ ]:
#creating a new data where the target is revenue : class 1 :revenue higher than 0 class 0 :revenue=0
zero_revenu = len(data[data.revenue==0])
def revenue(number):
    if number==0:
        return 0
    else:
        return 1
    
new_data['revenue'] = new_data['revenue'].map(revenue)

In [ ]:
#what the new data looks like :
new_data.sample(10)

In [ ]:
#Each class ratio in the data :
class_1 = len(new_data[new_data.revenue==1])/len(new_data)
class_0 = len(new_data[new_data.revenue==0])/len(new_data)

In [ ]:
#the ratio of class 0 over class 1, which indicat how much elemants from class0 we have for each element of class1
class_0/class_1

The ratio of class 0 over class 1, which indicat how much elemants from class0 we have for each element of class1
class_0/class_1

In [ ]:
#Droping all missing values
new_data.dropna(axis=0,inplace=True)

In [ ]:
#factorization of string values into numerical ones
new_data.user_device = pd.factorize(new_data.user_device)[0]
new_data.user_operating_system = pd.factorize(new_data.user_operating_system)[0]

In [ ]:
new_data.sample(5)

In [ ]:
#reset index
new_data.reset_index(inplace=True)
new_data.drop(columns=["index"],inplace=True)

In [ ]:
#Our classifier, we set class_weight to balanced so that the two classes will have the same total weight and thereby we deal 
#with the issue of imbalance data.
clf = XGBClassifier(base_score=class_1/class_0,random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)

In [ ]:
#our input and output vectors
X,y = np.array(new_data.drop(columns=["revenue"])),np.array(new_data.revenue)

In [ ]:
#train test split : 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
#The fit operation
clf.fit(X_train,y_train)

In [ ]:
#our prediction on the test data
y_pred = clf.predict(X_test)

In [ ]:
#our accuracy: we use a balnced accuracy in order to deal with imbalance data issues
balanced_accuracy_score(y_test,y_pred)

In [ ]:
# now we use a Kcross validation but preserving the same class ratio for each folder
skf = StratifiedKFold(n_splits=4,random_state=0)
scores = []
clf = XGBClassifier(random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = clf.fit(X_train, y_train.ravel())
    predictions = clf.predict(X_test)
    scores.append(balanced_accuracy_score(y_test, predictions))

In [ ]:
#our score :
np.mean(scores)

###### Now we will normalize the input vector hoping to find more robust result, then we will use logistic regression as a classifier.

In [ ]:
x = new_data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
new_data = pd.DataFrame(x_scaled,columns=data.columns)

In [ ]:
#our input and output vectors
X,y = np.array(new_data.drop(columns=["revenue"])),np.array(new_data.revenue)
#train test split : 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
clf = LogisticRegression(solver='lbfgs',class_weight="balanced",random_state=0)
clf = clf.fit(X_train,y_train)
#our prediction on the test data
y_pred = clf.predict(X_test)
#our accuracy using a balnced accuracy so that we deal with imbalance data issues
balanced_accuracy_score(y_test,y_pred)

In [ ]:
# now we use a Kcross validation but precerving the same class ratio for each folder
skf = StratifiedKFold(n_splits=4,random_state=0)
scores = []
clf = LogisticRegression(solver='lbfgs',class_weight="balanced",random_state=0)
#clf = XGBClassifier(random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = clf.fit(X_train, y_train.ravel())
    predictions = clf.predict(X_test)
    scores.append(balanced_accuracy_score(y_test, predictions))
#our score : 
np.mean(scores)

We had approximately the same result.

### Phase 2 : Simple imputers

Now, we will replace the missing values by numbers, we will use a lot of imputation techniques to do that.

In [ ]:
# our data
new_data = data.copy()
#factorization of string values into numerical ones
new_data.user_device = pd.factorize(new_data.user_device)[0]
new_data.user_operating_system = pd.factorize(new_data.user_operating_system)[0] 
imputation_data = new_data.copy()#a copy of the current data
imputation_data['revenue'] = imputation_data['revenue'].map(revenue)

We will use three simple imputation techniques: mean, median and most_frequent

In [ ]:
#Imputers
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_median= SimpleImputer(missing_values=np.nan, strategy='median')
imp_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
#the resulting column:
mean_strategy = imp_mean.fit_transform(np.array(imputation_data.average_seconds_played).reshape(-1,1))
median_strategy = imp_median.fit_transform(np.array(imputation_data.average_seconds_played).reshape(-1,1))
frequent_strategy = imp_frequent.fit_transform(np.array(imputation_data.average_seconds_played).reshape(-1,1))

In [ ]:
##using the mean as strategy:
imputation_data.average_seconds_played = mean_strategy
clf = XGBClassifier(random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)
X,y = np.array(imputation_data.drop(columns=["revenue"])),np.array(imputation_data.revenue).astype("int")
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("using mean as stratrgy for emputer, the balanced accuracy is: ",balanced_accuracy_score(y_test,y_pred))

In [ ]:
##using the median as strategy:
imputation_data.average_seconds_played = median_strategy
clf = XGBClassifier(random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)
X,y = np.array(imputation_data.drop(columns=["revenue"])),np.array(imputation_data.revenue)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("using median as stratrgy for emputer, the balanced accuracy is: ",balanced_accuracy_score(y_test,y_pred))

In [ ]:
##using the most_frequent as strategy:
imputation_data.average_seconds_played = frequent_strategy
clf = XGBClassifier(random_state=0,class_weight="balanced",max_depth=10,n_estimators=100)
X,y = np.array(imputation_data.drop(columns=["revenue"])),np.array(imputation_data.revenue)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("using most_frequent as stratrgy for emputer, the balanced accuracy is: ",balanced_accuracy_score(y_test,y_pred))

#### As we can see, there is no improvement in the result, in the contrary the accuracy has been reduced by 3%, this is reasonable since the methods we used are really simple and may not match the real features distribution

### Other metric : 

In [ ]:
def Performance(Model,Y,X):
    fpr, tpr, _ = roc_curve(Y, Model.predict_proba(X)[:,1])
    AUC  = auc(fpr, tpr)
    print ('the AUC is : %0.4f' %  AUC)
    plt.figure(figsize=(10,7))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % AUC)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
Performance(clf,y_test,X_test)

# <a style="color:#920037"> 3. SUBMISSION ON RAMP </a>

## <a style="color:#920037"> 3.1 Submission files  </a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_df.user_device = pd.factorize(X_df.user_device)[0]
        X_df.user_operating_system = pd.factorize(X_df.user_operating_system)[0]
        X_df.reset_index(inplace=True)
        X_df.drop(columns=["index"],inplace=True)
        imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
        mean_strategy = imp_mean.fit_transform(np.array(X_df.average_seconds_played).reshape(-1,1))
        X_df.average_seconds_played = mean_strategy
        X = np.array(X_df)
        return X

In [ ]:
from sklearn.ensemble  import RandomForestClassifier
from sklearn.base import BaseEstimator


class Classifier(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestClassifier(class_weight="balanced")

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict_proba(self, X):
        return self.reg.predict_proba(X)

## <a style="color:#920037"> 3.2 Local testing before submitted to RAMP  </a>

You have to test your submissions files before doing your submissions. In that goal, use ramp_test_submission command. Notice that this unit test run in the folder [`submissions/starting_kit`](/tree/submissions/starting_kit).
Before running the test, make sure you have done the following tasks:

- install ramp-workflow
- write python files classifier.py  and feature_extractor.py and put it in the following folder [`submissions/starting_kit`].
- download the data by excuting `python download_data.py`

You have two possibilities to test your submissions, a complete test (train and test the model with cross validtion like the server will do or a quick test (just to make sure the submissions code are working). If the test run, print train and test erros on each fold of the cross validation you can then submit to ramp.

Execute the following cell to do a quick test:

In [ ]:
!ramp_test_submission --quick-test